In [2]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from collections import defaultdict

# Пример корпуса текстов
corpus = ["Кошка ловит мышку", "Собака гоняет кота", "Мышка бежит от кошки и собаки"]

# Обучение модели Word2Vec на корпусе текстов
tokenized_corpus = [word_tokenize(text.lower()) for text in corpus]
model = Word2Vec(
    sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, sg=1
)

# Создаем обратный индекс с учетом векторного представления слов
index = defaultdict(list)
for idx, text in enumerate(corpus):
    words = word_tokenize(text.lower())
    for word in words:
        index[word].append(idx)


# Функция поиска по обратному индексу с использованием векторного представления слов
def search(query):
    query_words = word_tokenize(query.lower())
    query_vec = sum([model.wv[word] for word in query_words]) / len(query_words)

    result_docs = set(index[query_words[0]])
    for word in query_words[1:]:
        result_docs &= set(index[word])

    return list(result_docs)


# Пример запроса
query = "кошка мышка"
result = search(query)

if result:
    print(f"Результаты поиска для запроса '{query}':")
    for doc_idx in result:
        print(corpus[doc_idx])
else:
    print("Ничего не найдено")

Ничего не найдено
